# Crea estructura de datos pares de relays


In [22]:
import json
import numpy as np
import os
import copy # Para crear copias independientes de los diccionarios de relés

# --- Constantes para el cálculo de Time_out ---
K = 0.14
N = 0.02
DECIMAL_PLACES = 4

# --- Rutas de los archivos ---
data_coordination_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/data_coordination.json"
# Cambiado según solicitud: apunta al archivo (presumiblemente renombrado) 'relay_values.json'
relay_values_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values.json"
# El archivo de salida contendrá la LISTA de pares con la nueva estructura
output_pairs_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json" # Nuevo nombre para evitar sobreescribir

# --- Función para eliminar timestamps recursivamente ---
def eliminar_timestamp(obj):
    if isinstance(obj, dict):
        obj.pop("timestamp", None)
        for valor in obj.values():
            eliminar_timestamp(valor)
    elif isinstance(obj, list):
        for elemento in obj:
            eliminar_timestamp(elemento)

# --- Función para calcular el tiempo de operación (Time_out) ---
def calculate_operation_time(I_shc, I_pi, TDS):
    # (Sin cambios en la función de cálculo)
    if not all(isinstance(x, (int, float)) for x in [I_shc, I_pi, TDS]): return 0.0
    if any(x is None or x <= 0 for x in [I_pi, TDS]): return 0.0
    if I_shc < 0: return 0.0
    if I_shc == 0: return 0.0
    try:
        if abs(I_pi) < 1e-9: return 0.0
        M = I_shc / I_pi
    except ZeroDivisionError: return 0.0
    if M <= 1: return 0.0
    try:
        denominator = M**N - 1
        if abs(denominator) < 1e-9: return 0.0
        timeout = (K / denominator) * TDS
        return round(timeout, DECIMAL_PLACES) if np.isfinite(timeout) else 0.0
    except (OverflowError, ValueError): return 0.0
    except Exception: return 0.0


# --- Procesamiento Principal ---
relay_pairs_list = []
processed_scenarios_base = {}

try:
    # 1. Cargar y pre-procesar los datos base de coordinación
    print(f"Cargando datos base desde: {data_coordination_file}")
    with open(data_coordination_file, 'r') as archivo:
        datos_coordinacion = json.load(archivo)

    for escenario_base in datos_coordinacion:
        scenario_id = escenario_base.get("scenario_id")
        if scenario_id:
            escenario_procesado = escenario_base.copy()
            escenario_procesado.pop("_id", None)
            escenario_procesado.pop("scenario_id", None)
            eliminar_timestamp(escenario_procesado)
            processed_scenarios_base[scenario_id] = escenario_procesado
    print(f"Datos base cargados y pre-procesados para {len(processed_scenarios_base)} escenarios.")

    # 2. Cargar los valores de los relés (antes 'optimizados')
    print(f"Cargando valores de relés desde: {relay_values_file}")
    with open(relay_values_file, 'r') as archivo:
        # Renombrado: datos_relay_values
        datos_relay_values = json.load(archivo)
    # Renombrado: relay_values_map
    # ASUNCIÓN: La clave DENTRO del JSON ahora es 'relay_values'.
    # Si sigue siendo 'optimized_relay_values', cambia el .get() abajo.
    relay_values_map = {esc.get("scenario_id"): esc.get("relay_values", {})
                        for esc in datos_relay_values if esc.get("scenario_id")}
    print(f"Valores de relés cargados para {len(relay_values_map)} escenarios.")

    # 3. Fusionar, calcular timeouts y CONSTRUIR LISTA DE PARES (con línea en main_relay)
    print("Procesando escenarios para crear pares de relés independientes...")
    pair_count = 0
    for scenario_id, escenario_data in processed_scenarios_base.items():
        print(f"Procesando escenario: {scenario_id}")
        # Renombrado: current_relay_values
        current_relay_values = relay_values_map.get(scenario_id)

        if not current_relay_values:
            print(f"  Advertencia: No se encontraron valores de relé para {scenario_id}. Omitiendo pares de este escenario.")
            continue

        # Iterar sobre las líneas (como "L1-2")
        for linea_key, linea_data in escenario_data.items():
             if isinstance(linea_data, dict) and 'scenarios' in linea_data:
                # Iterar sobre los escenarios internos/fallas (como "90")
                for fault_key, internal_scenario_data in linea_data.get('scenarios', {}).items():
                    main_relay_info_orig = internal_scenario_data.get('main')
                    backups_orig = internal_scenario_data.get('backups', [])

                    if isinstance(main_relay_info_orig, dict) and isinstance(backups_orig, list):
                        # Procesar relé principal UNA VEZ
                        main_relay_info = copy.deepcopy(main_relay_info_orig)
                        main_relay_name = main_relay_info.get('relay')
                        main_time_out = 0.0

                        # --- Añadir la línea principal AL OBJETO main_relay ---
                        main_relay_info['line'] = linea_key
                        # ----------------------------------------------------

                        # Renombrado: current_relay_values
                        if main_relay_name and main_relay_name in current_relay_values:
                            # Renombrado: relay_setting
                            relay_setting = current_relay_values[main_relay_name]
                            main_relay_info['TDS'] = relay_setting.get('TDS')
                            main_relay_info['pick_up'] = relay_setting.get('pickup')
                            ishc = main_relay_info.get('Ishc')
                            pickup = main_relay_info.get('pick_up')
                            tds = main_relay_info.get('TDS')
                            main_time_out = calculate_operation_time(ishc, pickup, tds)
                            main_relay_info['Time_out'] = main_time_out
                        else:
                             ishc = main_relay_info.get('Ishc')
                             pickup = main_relay_info.get('pick_up')
                             tds = main_relay_info.get('TDS')
                             main_time_out = calculate_operation_time(ishc, pickup, tds)
                             main_relay_info['Time_out'] = main_time_out

                        # Iterar sobre CADA relé de respaldo
                        for backup_relay_info_orig in backups_orig:
                            if isinstance(backup_relay_info_orig, dict):
                                backup_relay_info = copy.deepcopy(backup_relay_info_orig)
                                backup_relay_name = backup_relay_info.get('relay')
                                backup_time_out = 0.0

                                # Renombrado: current_relay_values
                                if backup_relay_name and backup_relay_name in current_relay_values:
                                    # Renombrado: relay_setting
                                    relay_setting = current_relay_values[backup_relay_name]
                                    backup_relay_info['TDS'] = relay_setting.get('TDS')
                                    backup_relay_info['pick_up'] = relay_setting.get('pickup')
                                    ishc = backup_relay_info.get('Ishc')
                                    pickup = backup_relay_info.get('pick_up')
                                    tds = backup_relay_info.get('TDS')
                                    backup_time_out = calculate_operation_time(ishc, pickup, tds)
                                    backup_relay_info['Time_out'] = backup_time_out
                                else:
                                    ishc = backup_relay_info.get('Ishc')
                                    pickup = backup_relay_info.get('pick_up')
                                    tds = backup_relay_info.get('TDS')
                                    backup_time_out = calculate_operation_time(ishc, pickup, tds)
                                    backup_relay_info['Time_out'] = backup_time_out

                                # Crear el diccionario del par (SIN 'line' de nivel superior)
                                pair_entry = {
                                    "scenario_id": scenario_id,
                                    # "line": linea_key, # <-- Eliminado de aquí
                                    "fault": fault_key,
                                    "main_relay": main_relay_info, # Ya contiene 'line'
                                    "backup_relay": backup_relay_info # Mantiene su 'line' interna opcional
                                }
                                relay_pairs_list.append(pair_entry)
                                pair_count += 1

    print(f"Procesamiento completado. Se generaron {pair_count} pares de relés.")

    # 4. Guardar la LISTA de pares en el archivo de salida
    print(f"Guardando lista de pares en: {output_pairs_file}")
    output_dir = os.path.dirname(output_pairs_file)
    if output_dir: os.makedirs(output_dir, exist_ok=True)

    with open(output_pairs_file, 'w') as f:
        json.dump(relay_pairs_list, f, indent=2)

    print("¡Proceso completado exitosamente!")

except FileNotFoundError as e:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo: {e.filename}")
    print("Verifica que las rutas y nombres de archivo sean correctos (especialmente 'relay_values.json').")
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo JSON está mal formado: {e}")
except KeyError as e:
    print(f"Error: Falta una clave esperada en los datos: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    traceback.print_exc()

Cargando datos base desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/data_coordination.json
Datos base cargados y pre-procesados para 68 escenarios.
Cargando valores de relés desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/optimized_relay_values.json
Valores de relés cargados para 68 escenarios.
Procesando escenarios para crear pares de relés independientes...
Procesando escenario: scenario_1
Procesando escenario: scenario_2
Procesando escenario: scenario_3
Procesando escenario: scenario_4
Procesando escenario: scenario_5
Procesando escenario: scenario_6
Procesando escenario: scenario_7
Procesando escenario: scenario_8
Procesando escenario: scenario_9
Procesando escenario: scenario_10
Procesando escenario: scenario_11
Procesando escenario: scenario_12
Procesando escenario: scenario_13
Procesando escenario: scenario_14
Procesando escenario: scenario_15
Procesando escenario: scenario_16
Procesando escenario: scenario_17
Proce

# Filtrar por pares buenos


In [18]:
import json

# Ruta de tu archivo original
input_path  = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs.json'
# Ruta donde quieres guardar el filtrado
output_path = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_best.json'

# 1. Cargar JSON original
with open(input_path, 'r') as f:
    data = json.load(f)

# 2. Lista de escenarios considerados “Buenos”
buenos = [
    "scenario_29", "scenario_4", "scenario_24", "scenario_31", "scenario_36", 
    "scenario_25", "scenario_55", "scenario_26", "scenario_30", "scenario_21", 
    "scenario_57", "scenario_56", "scenario_34", "scenario_54", "scenario_23", 
    "scenario_63", "scenario_58", "scenario_1",  "scenario_33", "scenario_66", 
    "scenario_22", "scenario_59", "scenario_28", "scenario_67", "scenario_53", 
    "scenario_60", "scenario_27", "scenario_32", "scenario_35", "scenario_68", 
    "scenario_62", "scenario_61", "scenario_64", "scenario_65"
]

# 3. Filtrar sólo las entradas cuyo 'scenario_id' esté en la lista de buenos
filtered = [item for item in data if item.get('scenario_id') in buenos]

# 4. Guardar el resultado en el nuevo JSON
with open(output_path, 'w') as f:
    json.dump(filtered, f, indent=2)

print(f'Se han guardado {len(filtered)} pares en:\n{output_path}')


Se han guardado 3400 pares en:
/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_best.json


# Eliminar TDS & Pickup

In [23]:
import json
import os
import copy # Usaremos deepcopy para asegurar que no modificamos el original

# --- Rutas de los archivos ---
# Archivo de entrada es la lista de pares generada anteriormente
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json"

# Nombre del archivo de salida final según tu solicitud
output_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json"

print(f"Archivo de entrada: {input_file}")
print(f"Archivo de salida: {output_file}")

try:
    # 1. Cargar el archivo JSON existente (que es una LISTA)
    print("Cargando datos (lista de pares) desde el archivo de entrada...")
    with open(input_file, 'r') as f:
        # Ahora 'data' será una lista de diccionarios
        data = json.load(f)
    print("Datos cargados correctamente.")

    # Verificar que los datos cargados son una lista
    if not isinstance(data, list):
        raise TypeError(f"Error: El archivo de entrada {input_file} no contiene una lista JSON como se esperaba.")

    # Crear una copia profunda para modificarla
    modified_data = copy.deepcopy(data)

    # 2. Iterar sobre la LISTA y eliminar las claves 'pick_up' y 'TDS' de cada par
    print("Procesando datos para eliminar 'pick_up' y 'TDS' de cada par...")
    pairs_processed = 0
    relays_modified_count = 0 # Contará cuántos diccionarios (main o backup) fueron modificados

    # Iterar directamente sobre cada diccionario 'pair_entry' en la lista 'modified_data'
    for pair_entry in modified_data:
        if isinstance(pair_entry, dict):
            # Procesar el diccionario del relé principal
            main_relay_info = pair_entry.get('main_relay')
            if isinstance(main_relay_info, dict):
                # Eliminar claves si existen, si no, no hacer nada
                # pop devuelve el valor eliminado o None, is not None verifica si se eliminó algo
                key_removed_main = main_relay_info.pop('pick_up', None) is not None
                key_removed_main = main_relay_info.pop('TDS', None) is not None or key_removed_main # Se actualiza si alguna de las dos se eliminó
                if key_removed_main:
                    relays_modified_count += 1

            # Procesar el diccionario del relé de respaldo
            backup_relay_info = pair_entry.get('backup_relay')
            if isinstance(backup_relay_info, dict):
                # Eliminar claves si existen
                key_removed_backup = backup_relay_info.pop('pick_up', None) is not None
                key_removed_backup = backup_relay_info.pop('TDS', None) is not None or key_removed_backup
                if key_removed_backup:
                    relays_modified_count += 1

            pairs_processed += 1 # Incrementar el contador de pares procesados
        else:
            print(f"Advertencia: Se encontró un elemento no diccionario en la lista: {pair_entry}")


    print(f"Procesamiento completado. Se procesaron {pairs_processed} pares.")
    print(f"Se modificaron (eliminando pick_up/TDS de main o backup) {relays_modified_count} diccionarios de relé.")

    # 3. Guardar los datos modificados (la lista) en un nuevo archivo JSON
    print(f"Guardando datos modificados en: {output_file}")
    # Crear directorio si no existe (opcional pero buena práctica)
    output_dir = os.path.dirname(output_file)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_file, 'w') as f:
        # Guardar la lista modificada directamente
        json.dump(modified_data, f, indent=2)

    print("¡Archivo guardado exitosamente!")

except FileNotFoundError:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
    print("Por favor, verifica que la ruta y el nombre del archivo sean correctos.")
except TypeError as e:
    print(e) # Imprimir el mensaje de error si el archivo no es una lista
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    print("--- Traceback ---")
    traceback.print_exc()
    print("-----------------")

Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_bad.json
Archivo de salida: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json
Cargando datos (lista de pares) desde el archivo de entrada...
Datos cargados correctamente.
Procesando datos para eliminar 'pick_up' y 'TDS' de cada par...
Procesamiento completado. Se procesaron 3400 pares.
Se modificaron (eliminando pick_up/TDS de main o backup) 6732 diccionarios de relé.
Guardando datos modificados en: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json
¡Archivo guardado exitosamente!


# Update Inom

In [24]:
import json
import os
# No necesitamos copy esta vez, modificaremos la estructura cargada directamente

# --- Rutas de los archivos ---
load_flow_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/load_flow_results.json"
# El archivo que contiene los pares de relés (con 'line' DENTRO de 'main_relay')
# y que será MODIFICADO
# ASUNCIÓN: Este archivo es el resultado del script anterior (ej. 'independent_relay_pairs_v2.json')
#           El usuario lo llama 'input_data.json' en el prompt, así que usamos esa ruta.
#           ¡Asegúrate de que este archivo tenga la estructura correcta!
relay_pairs_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json"


print(f"Archivo de resultados de flujo de carga: {load_flow_file}")
print(f"Archivo de pares de relés (entrada y salida): {relay_pairs_file}")
print("NOTA: Se espera que la clave 'line' esté DENTRO del objeto 'main_relay' en el archivo de entrada.")

# Diccionario para almacenar las corrientes por escenario y línea para búsqueda rápida
load_flow_map = {}

try:
    # 1. Cargar y pre-procesar los resultados del flujo de carga
    print("Cargando y pre-procesando resultados de flujo de carga...")
    with open(load_flow_file, 'r') as f:
        load_flow_data = json.load(f)

    if not isinstance(load_flow_data, list):
        raise TypeError(f"Error: El archivo {load_flow_file} no contiene una lista JSON.")

    for scenario_result in load_flow_data:
        scenario_id = scenario_result.get("scenario_id")
        lines_data = scenario_result.get("lines")

        if scenario_id and isinstance(lines_data, list):
            line_currents = {}
            for line_info in lines_data:
                line_name = line_info.get("name")
                current_a = line_info.get("current_a")
                if line_name and current_a is not None:
                    line_currents[line_name] = current_a
            load_flow_map[scenario_id] = line_currents

    print(f"Resultados de flujo de carga pre-procesados para {len(load_flow_map)} escenarios.")
    if not load_flow_map:
        print("Advertencia: No se cargaron datos válidos del flujo de carga. El archivo de salida no se modificará.")
        exit()

    # 2. Cargar el archivo de pares de relés (con la nueva estructura)
    print(f"Cargando datos de pares de relés desde: {relay_pairs_file}")
    with open(relay_pairs_file, 'r') as f:
        relay_pairs_data = json.load(f)

    if not isinstance(relay_pairs_data, list):
         raise TypeError(f"Error: El archivo {relay_pairs_file} no contiene una lista JSON de pares.")
    print("Datos de pares de relés cargados.")

    # 3. Iterar sobre los pares y añadir la corriente 'current_a'
    print("Añadiendo valores de 'current_a' a los pares de relés...")
    pairs_processed = 0
    currents_added_count = 0
    missing_currents_warnings = 0

    for pair_entry in relay_pairs_data:
        pairs_processed += 1 # Contar cada par leído
        if not isinstance(pair_entry, dict):
            print(f"Advertencia: Se encontró un elemento no diccionario en {relay_pairs_file}. Omitido.")
            continue

        scenario_id = pair_entry.get("scenario_id")
        main_relay_info = pair_entry.get("main_relay")
        backup_relay_info = pair_entry.get("backup_relay")

        # --- Obtener Main Line Name desde DENTRO de main_relay_info ---
        main_line_name = None
        if isinstance(main_relay_info, dict):
            main_line_name = main_relay_info.get("line") # Busca la clave 'line' aquí
        # -------------------------------------------------------------

        if not scenario_id:
            print(f"Advertencia: Par omitido por falta de 'scenario_id': {pair_entry}")
            continue

        scenario_currents = load_flow_map.get(scenario_id)
        if not scenario_currents:
            # print(f"Advertencia: No se encontraron datos de flujo de carga para scenario_id='{scenario_id}'. Asignando 'null' a 'current_a'.")
            if isinstance(main_relay_info, dict): main_relay_info['current_a'] = None
            if isinstance(backup_relay_info, dict): backup_relay_info['current_a'] = None
            missing_currents_warnings += 1
            continue

        # --- Añadir corriente al relé principal ---
        if isinstance(main_relay_info, dict):
            if main_line_name: # Solo proceder si obtuvimos un nombre de línea
                current_a_main = scenario_currents.get(main_line_name)
                if current_a_main is not None:
                    main_relay_info['current_a'] = current_a_main
                    currents_added_count += 1
                else:
                    main_relay_info['current_a'] = None # Se buscó línea pero no corriente
                    missing_currents_warnings += 1
                    # print(f"Advertencia: No se encontró 'current_a' para scenario='{scenario_id}', línea principal='{main_line_name}'.")
            else:
                main_relay_info['current_a'] = None # No se encontró 'line' en main_relay
                missing_currents_warnings += 1
                # print(f"Advertencia: No se encontró clave 'line' en main_relay para scenario='{scenario_id}'.")
        # Si main_relay_info no es dict, no se hace nada.

        # --- Añadir corriente al relé de respaldo ---
        if isinstance(backup_relay_info, dict):
            # Determinar línea de respaldo: la suya propia o la principal (que puede ser None)
            backup_line_name = backup_relay_info.get('line', main_line_name)

            if backup_line_name: # Si se pudo determinar un nombre de línea
                current_a_backup = scenario_currents.get(backup_line_name)
                if current_a_backup is not None:
                    backup_relay_info['current_a'] = current_a_backup
                    currents_added_count += 1
                else:
                    backup_relay_info['current_a'] = None # Se buscó línea pero no corriente
                    missing_currents_warnings += 1
                    # print(f"Advertencia: No se encontró 'current_a' para scenario='{scenario_id}', línea respaldo='{backup_line_name}'.")
            else:
                 backup_relay_info['current_a'] = None # No se pudo determinar nombre de línea
                 missing_currents_warnings += 1
                 # print(f"Advertencia: No se pudo determinar línea para backup_relay en scenario='{scenario_id}'.")
        # Si backup_relay_info no es dict, no se hace nada.


    print(f"Procesamiento completado.")
    print(f"Se procesaron {pairs_processed} pares de relés del archivo.")
    print(f"Se añadieron/actualizaron valores de 'current_a' a {currents_added_count} entradas de relé (main o backup).")
    if missing_currents_warnings > 0:
        print(f"Hubo {missing_currents_warnings} casos donde no se encontró la clave 'line' o el valor 'current_a' correspondiente (se asignó 'null').")

    # 4. Guardar los datos modificados DE VUELTA al archivo original
    print(f"Guardando datos actualizados en: {relay_pairs_file}")
    with open(relay_pairs_file, 'w') as f:
        json.dump(relay_pairs_data, f, indent=2)

    print(f"¡Archivo {relay_pairs_file} actualizado exitosamente!")

except FileNotFoundError as e:
    print(f"Error CRÍTICO: No se pudo encontrar un archivo necesario: {e.filename}")
    print("Verifica las rutas de 'load_flow_results.json' y 'input_data.json'.")
except TypeError as e:
    print(f"Error CRÍTICO: Problema con el tipo de datos esperado en un archivo JSON: {e}")
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: Un archivo JSON está mal formado: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    print("--- Traceback ---")
    traceback.print_exc()
    print("-----------------")

Archivo de resultados de flujo de carga: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/load_flow_results.json
Archivo de pares de relés (entrada y salida): /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json
NOTA: Se espera que la clave 'line' esté DENTRO del objeto 'main_relay' en el archivo de entrada.
Cargando y pre-procesando resultados de flujo de carga...
Resultados de flujo de carga pre-procesados para 68 escenarios.
Cargando datos de pares de relés desde: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json
Datos de pares de relés cargados.
Añadiendo valores de 'current_a' a los pares de relés...
Procesamiento completado.
Se procesaron 3400 pares de relés del archivo.
Se añadieron/actualizaron valores de 'current_a' a 6800 entradas de relé (main o backup).
Guardando datos actualizados en: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data

# crear json de referencia

In [21]:
import json
import os
import copy # Usaremos deepcopy para asegurar que no modificamos el original

# --- Rutas de los archivos ---
# Archivo de entrada es la lista de pares generada anteriormente
input_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_best.json"

# Nombre del archivo de salida final según tu solicitud
output_file = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/reference_data.json"

print(f"Archivo de entrada: {input_file}")
print(f"Archivo de salida: {output_file}")

try:
    # 1. Cargar el archivo JSON existente (que es una LISTA)
    print("Cargando datos (lista de pares) desde el archivo de entrada...")
    with open(input_file, 'r') as f:
        # Ahora 'data' será una lista de diccionarios
        data = json.load(f)
    print("Datos cargados correctamente.")

    # Verificar que los datos cargados son una lista
    if not isinstance(data, list):
        raise TypeError(f"Error: El archivo de entrada {input_file} no contiene una lista JSON como se esperaba.")

    # Crear una copia profunda para modificarla
    modified_data = copy.deepcopy(data)

    # 2. Iterar sobre la LISTA y eliminar las claves 'pick_up' y 'TDS' de cada par
    print("Procesando datos para dejar 'pick_up' y 'TDS' de cada par...")
    pairs_processed = 0
    relays_modified_count = 0 # Contará cuántos diccionarios (main o backup) fueron modificados

    # Iterar directamente sobre cada diccionario 'pair_entry' en la lista 'modified_data'
    for pair_entry in modified_data:
        if isinstance(pair_entry, dict):
            # Procesar el diccionario del relé principal
            main_relay_info = pair_entry.get('main_relay')
            if isinstance(main_relay_info, dict):
                # Eliminar claves si existen, si no, no hacer nada
                # pop devuelve el valor eliminado o None, is not None verifica si se eliminó algo
                key_removed_main = main_relay_info.pop('Ishc', None) is not None
                key_removed_main = main_relay_info.pop('Time_out', None) is not None or key_removed_main # Se actualiza si alguna de las dos se eliminó
                if key_removed_main:
                    relays_modified_count += 1

            # Procesar el diccionario del relé de respaldo
            backup_relay_info = pair_entry.get('backup_relay')
            if isinstance(backup_relay_info, dict):
                # Eliminar claves si existen
                key_removed_backup = backup_relay_info.pop('Time_out', None) is not None
                key_removed_backup = backup_relay_info.pop('Ishc', None) is not None or key_removed_backup
                if key_removed_backup:
                    relays_modified_count += 1

            pairs_processed += 1 # Incrementar el contador de pares procesados
        else:
            print(f"Advertencia: Se encontró un elemento no diccionario en la lista: {pair_entry}")


    print(f"Procesamiento completado. Se procesaron {pairs_processed} pares.")
    print(f"Se modificaron (eliminando pick_up/TDS de main o backup) {relays_modified_count} diccionarios de relé.")

    # 3. Guardar los datos modificados (la lista) en un nuevo archivo JSON
    print(f"Guardando datos modificados en: {output_file}")
    # Crear directorio si no existe (opcional pero buena práctica)
    output_dir = os.path.dirname(output_file)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_file, 'w') as f:
        # Guardar la lista modificada directamente
        json.dump(modified_data, f, indent=2)

    print("¡Archivo guardado exitosamente!")

except FileNotFoundError:
    print(f"Error CRÍTICO: No se pudo encontrar el archivo de entrada: {input_file}")
    print("Por favor, verifica que la ruta y el nombre del archivo sean correctos.")
except TypeError as e:
    print(e) # Imprimir el mensaje de error si el archivo no es una lista
except json.JSONDecodeError as e:
    print(f"Error CRÍTICO: El archivo de entrada JSON ({input_file}) está mal formado: {e}")
except Exception as e:
    import traceback
    print(f"Error inesperado durante el procesamiento: {e}")
    print("--- Traceback ---")
    traceback.print_exc()
    print("-----------------")

Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_best.json
Archivo de salida: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/reference_data.json
Cargando datos (lista de pares) desde el archivo de entrada...
Datos cargados correctamente.
Procesando datos para dejar 'pick_up' y 'TDS' de cada par...
Procesamiento completado. Se procesaron 3400 pares.
Se modificaron (eliminando pick_up/TDS de main o backup) 6800 diccionarios de relé.
Guardando datos modificados en: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/reference_data.json
¡Archivo guardado exitosamente!
